In [31]:
import tensorflow as tf
from tensorflow_model_optimization.quantization.keras import quantize_model
from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer

In [32]:
# Load the existing TensorFlow Lite model
tflite_model_path = 'cow_weight_detection_model.tflite'  # Replace with the path to your .tflite model
tflite_model_content = open(tflite_model_path, 'rb').read()

In [33]:
# Create a TensorFlow Lite Interpreter
interpreter = tf.lite.Interpreter(model_content=tflite_model_content)
interpreter.allocate_tensors()

In [34]:
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [35]:
# Convert the TensorFlow Lite model to a TensorFlow Keras model
def lite_to_keras(interpreter):
    concrete_func = interpreter.signatures["serving_default"]
    concrete_func.inputs[0].set_shape([1, *concrete_func.inputs[0].shape[1:]])
    return concrete_func

In [36]:
# Create a TensorFlow Keras model
keras_model = tf.keras.Sequential()
keras_model.add(tf.keras.layers.InputLayer(input_shape=input_details[0]['shape'][1:]))

In [37]:
# Iterate through all tensors in the model
for layer_id, layer in enumerate(interpreter.get_tensor_details()):
    # Check if it's a Conv2D layer
    if 'conv' in layer['name']:
        filters = layer['shape'][-1]
        
        # Check if 'shape' exists and has more than one element
        if 'shape' in layer and layer['shape'][1:3].size > 0:
            kernel_size = tuple(layer['shape'][1:3])
        else:
            kernel_size = (3, 3)  
        
        activation = 'relu'  

        # Check if 'relu' is in the activation function, if not, assume 'linear'
        if 'relu' not in layer['name']:
            activation = 'linear'

        # Check if kernel_size has valid values
        if all(isinstance(val, int) for val in kernel_size):
            keras_model.add(tf.keras.layers.Conv2D(filters, kernel_size, activation=activation))
        else:
            # Manually set a default kernel size or choose a suitable size
            default_kernel_size = (3, 3)
            keras_model.add(tf.keras.layers.Conv2D(filters, default_kernel_size, activation=activation))

In [38]:
# Build the TensorFlow Keras model
keras_model.build(input_shape=(None, *input_details[0]['shape'][1:]))

In [39]:
# Annotate each layer for quantization
annotated_layers = [quantize_annotate_layer(layer) for layer in keras_model.layers]

In [40]:
# Build the annotated model before quantization
annotated_model.build(input_shape=(None, *input_details[0]['shape'][1:]))

In [41]:
# Quantize the TensorFlow Keras model
quantized_model = quantize_model(annotated_model)

In [42]:
# Convert the quantized model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpo1g5r6e5\assets


INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpo1g5r6e5\assets


In [43]:
# Save the quantized TFLite model to a file
tflite_filename = 'quantized_cow_weight_model.tflite'
with open(tflite_filename, 'wb') as f:
    f.write(quantized_tflite_model)